# Relatório Vigilância das Águas

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

#### Funções Auxiliares

In [5]:
def convert_string_to_datetime(date):
    return pd.to_datetime(date, format='%d/%m/%Y')


def read_excel_file(file_name):
    return pd.read_excel(file_name,
                         engine='openpyxl',
                         decimal=',',
                         converters={
                                       'Data da coleta': convert_string_to_datetime,
                                       'Data do laudo': convert_string_to_datetime,
                                       'Data de registro SISAGUA': convert_string_to_datetime,                                            
                                    })


def convert_string_to_nan(data_frame, word):
    return data_frame.replace(word, np.nan)


def drop_na_from_column(data_frame, column_name):
    return data_frame.dropna(subset=[column_name,])


def get_month_from_row(data_frame):
    return pd.to_datetime(data_frame['Data da coleta']).dt.month


def get_municipios_from_dataframe(data_frame):
    return data_frame['Município'].unique()


def get_municipios_invalidos(municipios, data_frame):
    return municipios[data_frame.groupby(['Município'])['mes_analise'].nunique() < 4]


def get_municipios_validos(municipios, data_frame):
    return municipios[data_frame.groupby(['Município'])['mes_analise'].nunique() >= 4]


def remove_municipios_from_dataframe(municipios_invalidos, data_frame):
    return data_frame[~data_frame['Município'].isin(municipios_invalidos)]


def group_by_fluoretacao_and_municipio(data_frame):
    return data_frame.groupby(['Município'])['Fluoreto(mg/L)'].value_counts()


def get_soma_fluoretacao(data_frame):
    return data_frame.groupby(['Município']).sum()


def get_municipios_with_valid_samples(data_frame, municipios_validos, municipios_soma):
    municipios_analises_validas = []
    for mun in municipios_validos:
        if data_frame[mun].get(0.0, 0) != municipios_soma[mun]:
            municipios_analises_validas.append(mun)
    return municipios_analises_validas


def get_data_frame_with_valid_samples(data_frame, municipios_analises_validas):
    return data_frame[data_frame['Município'].isin(municipios_analises_validas)]

#### Leitura dos arquivos excel

In [6]:
file_2020 = "./files/Relatório Vigilância das águas 01.01.2020 A 31.12.2020- SAA.xlsx"
#file_2021 = "./files/Relatório Vigilância das águas 01.01.2020 A 31.12.2020- SAA.xlsx"
#file_2022 = "./files/Relatório Vigilância das águas 01.01.2020 A 31.12.2020- SAA.xlsx"

output_2020 = "./files/limpo_2020.xlsx"

In [7]:
df_2020_raw = read_excel_file(file_name=file_2020)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [7]:
df_2020_raw.head()

,Município,Código IBGE,Motivo,Forma,Nome,Código,N° da amostra,Data da coleta,Data do laudo,Data de registro SISAGUA,...,E. coli,Cloro Residual Livre(mg/L),Cloro Residual Combinado (mg/L),Dióxido de Cloro (mg/L),Turbidez(uT),Fluoreto(mg/L),Fluoretação,Bactérias Heterotróficas(UFC/mL),Cor Aparente (uH),pH
0,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000002,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.33,NaN,NaN,0.90,0.8,Sim,Não realizada,Não realizada,6.50
1,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000001,2020-01-09,2020-01-10,2020-01-13,...,Ausente,2.15,NaN,NaN,0.81,0.8,Sim,Não realizada,Não realizada,6.50
2,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000003,2020-01-09,2020-01-10,2020-01-13,...,Ausente,0.94,NaN,NaN,1.10,Não realizada,Sim,Não realizada,Não realizada,6.50
3,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000004,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.27,NaN,NaN,0.69,0.8,Sim,Não realizada,Não realizada,6.50
4,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000006,2020-01-09,2020-01-10,2020-01-13,...,Ausente,2.26,NaN,NaN,0.84,0.8,Sim,Não realizada,Não realizada,6.50


In [8]:
df_2020_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38782 entries, 0 to 38781
Data columns (total 32 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Município                         38782 non-null  object        
 1   Código IBGE                       38782 non-null  int64         
 2   Motivo                            38782 non-null  object        
 3   Forma                             38782 non-null  object        
 4   Nome                              38782 non-null  object        
 5   Código                            38782 non-null  object        
 6   N° da amostra                     38782 non-null  object        
 7   Data da coleta                    38782 non-null  datetime64[ns]
 8   Data do laudo                     38782 non-null  datetime64[ns]
 9   Data de registro SISAGUA          38782 non-null  datetime64[ns]
 10  Procedência da coleta             38782 non-nu

## Limpeza dos dados

#### Conversão de **"Não realizada"** para **np.nan**

In [9]:
df_2020_no_nan = convert_string_to_nan(df_2020_raw, 'Não realizada')

In [10]:
df_2020_no_nan.head()

,Município,Código IBGE,Motivo,Forma,Nome,Código,N° da amostra,Data da coleta,Data do laudo,Data de registro SISAGUA,...,E. coli,Cloro Residual Livre(mg/L),Cloro Residual Combinado (mg/L),Dióxido de Cloro (mg/L),Turbidez(uT),Fluoreto(mg/L),Fluoretação,Bactérias Heterotróficas(UFC/mL),Cor Aparente (uH),pH
0,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000002,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.33,NaN,NaN,0.90,0.8,Sim,NaN,NaN,6.50
1,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000001,2020-01-09,2020-01-10,2020-01-13,...,Ausente,2.15,NaN,NaN,0.81,0.8,Sim,NaN,NaN,6.50
2,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000003,2020-01-09,2020-01-10,2020-01-13,...,Ausente,0.94,NaN,NaN,1.10,NaN,Sim,NaN,NaN,6.50
3,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000004,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.27,NaN,NaN,0.69,0.8,Sim,NaN,NaN,6.50
4,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000006,2020-01-09,2020-01-10,2020-01-13,...,Ausente,2.26,NaN,NaN,0.84,0.8,Sim,NaN,NaN,6.50


#### Remove linhas da coluna cujo valor de **"Fluoreto(mg/L)"** seja **np.nan**

In [13]:
df_2020 = drop_na_from_column(df_2020_no_nan, 'Fluoreto(mg/L)')

In [14]:
df_2020.head()

,Município,Código IBGE,Motivo,Forma,Nome,Código,N° da amostra,Data da coleta,Data do laudo,Data de registro SISAGUA,...,E. coli,Cloro Residual Livre(mg/L),Cloro Residual Combinado (mg/L),Dióxido de Cloro (mg/L),Turbidez(uT),Fluoreto(mg/L),Fluoretação,Bactérias Heterotróficas(UFC/mL),Cor Aparente (uH),pH
0,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000002,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.33,NaN,NaN,0.90,0.8,Sim,NaN,NaN,6.50
1,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000001,2020-01-09,2020-01-10,2020-01-13,...,Ausente,2.15,NaN,NaN,0.81,0.8,Sim,NaN,NaN,6.50
3,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000004,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.27,NaN,NaN,0.69,0.8,Sim,NaN,NaN,6.50
4,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000006,2020-01-09,2020-01-10,2020-01-13,...,Ausente,2.26,NaN,NaN,0.84,0.8,Sim,NaN,NaN,6.50
6,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000005,2020-01-09,2020-01-10,2020-01-13,...,Ausente,1.06,NaN,NaN,1.02,0.8,Sim,NaN,NaN,6.50


#### Cria coluna com o valor do mês em que a coleta de Fluor foi feita 

In [16]:
df_2020['mes_analise'] = get_month_from_row(df_2020)

/tmp/ipykernel_7163/512164321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['mes_analise'] = get_month_from_row(df_2020)


In [17]:
df_2020.head()

,Município,Código IBGE,Motivo,Forma,Nome,Código,N° da amostra,Data da coleta,Data do laudo,Data de registro SISAGUA,...,Cloro Residual Livre(mg/L),Cloro Residual Combinado (mg/L),Dióxido de Cloro (mg/L),Turbidez(uT),Fluoreto(mg/L),Fluoretação,Bactérias Heterotróficas(UFC/mL),Cor Aparente (uH),pH,mes_analise
0,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000002,2020-01-09,2020-01-10,2020-01-13,...,1.33,NaN,NaN,0.90,0.8,Sim,NaN,NaN,6.50,1
1,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000001,2020-01-09,2020-01-10,2020-01-13,...,2.15,NaN,NaN,0.81,0.8,Sim,NaN,NaN,6.50,1
3,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000004,2020-01-09,2020-01-10,2020-01-13,...,1.27,NaN,NaN,0.69,0.8,Sim,NaN,NaN,6.50,1
4,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000006,2020-01-09,2020-01-10,2020-01-13,...,2.26,NaN,NaN,0.84,0.8,Sim,NaN,NaN,6.50,1
6,SALTO VELOSO,421540,Rotina,SAA,SAA SALTO VELOSO,S421540000001,200907000005,2020-01-09,2020-01-10,2020-01-13,...,1.06,NaN,NaN,1.02,0.8,Sim,NaN,NaN,6.50,1


#### Cria DataFrame com os nomes dos municípios

In [19]:
municipios = get_municipios_from_dataframe(df_2020)

#### Cria DataFrame com os nomes dos municípios com menos de 4 meses de análise

In [22]:
municipios_invalidos = get_municipios_invalidos(municipios, df_2020)

In [23]:
municipios_validos = get_municipios_validos(municipios, df_2020)

#### Remove os municípios inválidos

In [25]:
df_2020_municipios_validos = remove_municipios_from_dataframe(municipios_invalidos, df_2020)

### Somente para validação, remover após testes

In [52]:
# df_2020_municipios_validos.loc[df_2020['Município'] == 'BRUSQUE', 'Fluoreto(mg/L)'] = 0.0

#### Conta as taxas de Fluoreto por município

#### Agrupa por municípios e fluoretação

In [27]:
df_2020_fluoreto = group_by_fluoretacao_and_municipio(df_2020_municipios_validos)
df_2020_fluoreto['IMARUI']

Fluoreto(mg/L)
0    1
Name: count, dtype: int64

In [29]:
municipios_soma = get_soma_fluoretacao(df_2020_fluoreto)
municipios_soma

Município
ABDON BATISTA    49
ABELARDO LUZ     46
AGROLANDIA       55
AGRONOMICA       55
AGUA DOCE        55
                 ..
WITMARSUM        49
XANXERE          83
XAVANTINA        82
XAXIM            59
ZORTEA           24
Name: count, Length: 287, dtype: int64

#### Remove municípos que possuam mais de 50% das amostras com valor igual a zero

In [31]:
municipios_analises_validas = get_municipios_with_valid_samples(df_2020_fluoreto, municipios_validos, municipios_soma)
# municipios_analises_validas        

In [33]:
df_2020_final = get_data_frame_with_valid_samples(df_2020_municipios_validos, municipios_analises_validas)
# df_2020_final = df_2020_municipios_validos[df_2020_municipios_validos['Município'].isin(municipios_analises_validas)]

In [58]:
df_2020_final.to_excel(output_2020) 